Tensor Field Networks

Implementation of shape classification demonstration

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as anim
import tensorflow as tf
import random
from math import pi, sqrt
import tensorfieldnetworks.utils as utils

from tensorfieldnetworks.ShapeClassificationModel import ShapeClassificationModel

tetris = [[(0, 0, 0), (0, 0, 1), (1, 0, 0), (1, 1, 0)],  # chiral_shape_1
          [(0, 0, 0), (0, 0, 1), (1, 0, 0), (1, -1, 0)], # chiral_shape_2
          [(0, 0, 0), (1, 0, 0), (0, 1, 0), (1, 1, 0)],  # square
          [(0, 0, 0), (0, 0, 1), (0, 0, 2), (0, 0, 3)],  # line
          [(0, 0, 0), (0, 0, 1), (0, 1, 0), (1, 0, 0)],  # corner
          [(0, 0, 0), (0, 0, 1), (0, 0, 2), (0, 1, 0)],  # T
          [(0, 0, 0), (0, 0, 1), (0, 0, 2), (0, 1, 1)],  # zigzag
          [(0, 0, 0), (1, 0, 0), (1, 1, 0), (2, 1, 0)]]  # L

dataset = [np.array(points_, dtype='float32') for points_ in tetris]
num_classes = len(dataset)

In [8]:
model = ShapeClassificationModel(num_classes)
model(dataset[0])
model.summary()
model(dataset[1])

[{0: [<tf.Tensor 'input:0' shape=(4, 1, 1) dtype=float32>]}, <tf.Tensor 'input_1:0' shape=(4, 4, 4) dtype=float32>, <tf.Tensor 'input_2:0' shape=(4, 4, 3) dtype=float32>]
test
[{0: [<tf.Tensor 'input:0' shape=(4, 1, 1) dtype=float32>]}, <tf.Tensor 'input_1:0' shape=(4, 4, 4) dtype=float32>, <tf.Tensor 'input_2:0' shape=(4, 4, 3) dtype=float32>]
test
[{0: [<tf.Tensor 'input:0' shape=(4, 4, 1) dtype=float32>], 1: [<tf.Tensor 'input_1:0' shape=(4, 4, 3) dtype=float32>]}, <tf.Tensor 'input_2:0' shape=(4, 4, 4) dtype=float32>, <tf.Tensor 'input_3:0' shape=(4, 4, 3) dtype=float32>]
test
[{0: [<tf.Tensor 'input:0' shape=(4, 4, 1) dtype=float32>], 1: [<tf.Tensor 'input_1:0' shape=(4, 4, 3) dtype=float32>]}, <tf.Tensor 'input_2:0' shape=(4, 4, 4) dtype=float32>, <tf.Tensor 'input_3:0' shape=(4, 4, 3) dtype=float32>]
test
[{0: [<tf.Tensor 'input:0' shape=(4, 4, 1) dtype=float32>], 1: [<tf.Tensor 'input_1:0' shape=(4, 4, 3) dtype=float32>]}, <tf.Tensor 'input_2:0' shape=(4, 4, 4) dtype=float32>, 

<tf.Tensor: shape=(8,), dtype=float32, numpy=
array([ 0.7116172 ,  0.66414523, -0.04071363, -0.03242678, -0.5528971 ,
       -0.3408018 , -0.36797613,  0.3079396 ], dtype=float32)>

In [4]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1.e-3)

In [9]:
max_epochs = 200
print_freq = 10

#with tf.profiler.experimental.Profile('logdir'):
for epoch in range(max_epochs):    
    loss_sum = 0.
    for label, shape in enumerate(dataset):
        with tf.GradientTape() as tape:
            pred = model(shape, training=True)
            truth = tf.one_hot(label, num_classes)
            loss = tf.nn.softmax_cross_entropy_with_logits(labels=truth, logits=pred)
            loss_sum += loss
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(grads_and_vars=zip(grads, model.trainable_variables))
        
    if epoch % print_freq == 0:
        print("Epoch %d: validation loss = %.3f" % (epoch, loss_sum / num_classes))

Epoch 0: validation loss = 0.089
Epoch 10: validation loss = 0.089
Epoch 20: validation loss = 0.089
Epoch 30: validation loss = 0.089
Epoch 40: validation loss = 0.089
Epoch 50: validation loss = 0.089
Epoch 60: validation loss = 0.089
Epoch 70: validation loss = 0.089
Epoch 80: validation loss = 0.089
Epoch 90: validation loss = 0.089
Epoch 100: validation loss = 0.089
Epoch 110: validation loss = 0.089
Epoch 120: validation loss = 0.089
Epoch 130: validation loss = 0.089
Epoch 140: validation loss = 0.089
Epoch 150: validation loss = 0.089
Epoch 160: validation loss = 0.089
Epoch 170: validation loss = 0.089
Epoch 180: validation loss = 0.089
Epoch 190: validation loss = 0.089


In [14]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(dataset, [tf.one_hot(label, num_classes) for label in range(num_classes)], epochs=10)


ValueError: Data cardinality is ambiguous:
  x sizes: 4, 4, 4, 4, 4, 4, 4, 4
  y sizes: 8, 8, 8, 8, 8, 8, 8, 8
Please provide data which shares the same first dimension.

In [6]:
rng = np.random.RandomState()
test_set_size = 25
predictions = [list() for i in range(len(dataset))]

correct_predictions = 0
total_predictions = 0
for i in range(test_set_size):
    for label, shape in enumerate(dataset):
        rotation = utils.random_rotation_matrix(rng)
        rotated_shape = np.dot(shape, rotation)
        translation = np.expand_dims(np.random.uniform(low=-3., high=3., size=(3)), axis=0)
        translated_shape = rotated_shape + translation
        output_label = tf.argmax(model(rotated_shape))
        total_predictions += 1
        if output_label == label:
            correct_predictions += 1
print('Test accuracy: %f' % (float(correct_predictions) / total_predictions))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Test accuracy: 1.000000
